<a href="https://colab.research.google.com/github/Nagmashaik123/Section203-Group06/blob/main/Countries_Cities_Location.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
df=pd.read_csv(r"CovidVaccine.csv")
df


,TweetDate,TweetText,TweetAuthor,TweetLocation,AllHashtags
0,2021-03-26 23:59:59,I'm making spike protein now. Neat!\n#COVID19V...,kstalenshi,"St Louis, MO",['COVID19Vaccine']
1,2021-03-26 23:59:50,Leo Terrell's emotional reaction to kids' COVI...,timewill10909,United States,"['COVID19', 'CovidVaccine', 'MentalHealthAware..."
2,2021-03-26 23:59:33,RT @Despise45: Woohoo!!! Starting in-store #Co...,Teram323Tere,United States,['Covidvaccination']
3,2021-03-26 23:59:32,Me leaving the medical centre after receiving ...,Neilfax,"Brighouse, Calderdale","['CovidVaccine', 'AstraZeneca']"
4,2021-03-26 23:59:23,RT @vanmoofer: this is the Tesla of eBikes\n\n...,teslaccu,World,['emobility']
...,...,...,...,...,...
9995,2021-03-26 08:04:02,RT @GCPartridge: And amongst the @M0MSuk #bre...,makesmilk,"London, UK","['breastfeeding', 'CovidVaccine']"
9996,2021-03-26 08:04:00,From the registration process to getting #COVI...,drharshvardhan,Delhi,"['COVID19Vaccine', 'COVID19Vaccination']"
9997,2021-03-26 08:04:00,"‘Dawai Bhi, Kadhai Bhi’ - Adopt to the #NewNor...",PIBSrinagar,"Srinagar, Kashmir","['NewNormal', 'COVIDVaccine']"
9998,2021-03-26 08:03:43,RT @DDNewslive: More than 5.5 cr #COVID19Vacci...,MKRaichand_IND,भारत,['COVID19Vaccine']


In [3]:
!pip install geopy

In [4]:
import numpy as np
import re

# Reading emoji patterns which contains emoticons , symbols & pictographs, transport & map symbols , flags
emojisCollection = re.compile("["
                       u"\U0001F600-\U0001F64F"  
                       u"\U0001F300-\U0001F5FF"  
                       u"\U0001F680-\U0001F6FF"  
                       u"\U0001F1E0-\U0001F1FF"  
                       u"\U00002702-\U000027B0"
                       u"\U000024C2-\U0001F251"
                       "]+", flags=re.UNICODE)
df['TweetLocation'] = (df['TweetLocation'].str.replace(emojisCollection, '').str.strip())

df['TweetLocation'] = df.TweetLocation.str.split(', ').str.get(0)
df['TweetLocation'] = df['TweetLocation'].replace(np.nan, 'Unknown', regex=True)
df['TweetLocation']

0            St Louis
1       United States
2       United States
3           Brighouse
4               World
            ...      
9995           London
9996            Delhi
9997         Srinagar
9998             भारत
9999          Unknown
Name: TweetLocation, Length: 10000, dtype: object

In [5]:
from functools import partial
from geopy.geocoders import Nominatim
import re

countryList =[]
geolocator = Nominatim(user_agent="COVID-19 vaccination attitudes")
geocode = partial(geolocator.geocode, language="en")
# Function to map the country for the corresponding city location
def checkCountry(city):
  
  # passing the city through geocode api to return the location if city is not unknow
  if city != "Unknown":
    location = geocode(city)   
    if location is None: 
      country = "Unknown" 
    else:
      country = str(location.address).split(", ")[-1]   
    
  # If city is not known then setting country as Unknown        
  else:    
    country = "Unknown"   
  return country


In [6]:
cities=[]
countries=[]
cities = df['TweetLocation'].tolist()
for city in cities:
    try:
        countries.append(checkCountry(city))  #Calling the checkCountry function
    except:
        countries.append('Unknown')

In [7]:
# Zipping city and corresponding country together
dict_1=dict(zip(cities,countries))

In [8]:
df['Country']=df['TweetLocation'].map(dict_1)
df

,TweetDate,TweetText,TweetAuthor,TweetLocation,AllHashtags,Country
0,2021-03-26 23:59:59,I'm making spike protein now. Neat!\n#COVID19V...,kstalenshi,St Louis,['COVID19Vaccine'],United States
1,2021-03-26 23:59:50,Leo Terrell's emotional reaction to kids' COVI...,timewill10909,United States,"['COVID19', 'CovidVaccine', 'MentalHealthAware...",United States
2,2021-03-26 23:59:33,RT @Despise45: Woohoo!!! Starting in-store #Co...,Teram323Tere,United States,['Covidvaccination'],United States
3,2021-03-26 23:59:32,Me leaving the medical centre after receiving ...,Neilfax,Brighouse,"['CovidVaccine', 'AstraZeneca']",United Kingdom
4,2021-03-26 23:59:23,RT @vanmoofer: this is the Tesla of eBikes\n\n...,teslaccu,World,['emobility'],United Arab Emirates
...,...,...,...,...,...,...
9995,2021-03-26 08:04:02,RT @GCPartridge: And amongst the @M0MSuk #bre...,makesmilk,London,"['breastfeeding', 'CovidVaccine']",United Kingdom
9996,2021-03-26 08:04:00,From the registration process to getting #COVI...,drharshvardhan,Delhi,"['COVID19Vaccine', 'COVID19Vaccination']",India
9997,2021-03-26 08:04:00,"‘Dawai Bhi, Kadhai Bhi’ - Adopt to the #NewNor...",PIBSrinagar,Srinagar,"['NewNormal', 'COVIDVaccine']",India
9998,2021-03-26 08:03:43,RT @DDNewslive: More than 5.5 cr #COVID19Vacci...,MKRaichand_IND,भारत,['COVID19Vaccine'],India


In [9]:
df.to_csv('CovidVaccineWorldWide.csv')